<a href="https://colab.research.google.com/github/amilkh/cs230-fer/blob/datagen/fer2013_LATEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#tensorflow_version 1.x

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.lib.io import file_io

import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

%matplotlib inline

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


try:
    pydot.Dot.create(pydot.Dot())
except:
    print('pydot error')

In [4]:
print(tf.__version__)
print(keras.__version__)

1.15.0
2.2.5


In [0]:
EPOCHS = 20
BS = 128
DROPOUT_RATE = 0.4
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

In [6]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
#model.add(Dropout(DROPOUT_RATE))
model.add(Conv2D(32, (4, 4), activation='relu',padding='same',name="conv2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
#model.add(Dropout(DROPOUT_RATE))
model.add(Conv2D(64, (5, 5), activation='relu',padding='same',name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model.add(Flatten())
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(DROPOUT_RATE))
model.add(Dense(7, activation='softmax',name='fcsoftmax'))

#TODO: weight decay of 0.0001...initial learning rate is set to 0.01 and reduced by a factor of 2 at every 25 epoch
sgd = SGD(lr=SGD_LEARNING_RATE,momentum=0.9, decay=SGD_DECAY, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
#rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100)
rlrop = ReduceLROnPlateau(monitor='val_acc',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
def get_data(datas, bs= 32, aug=None):
    # Data preparation
    while True:
      file_stream = file_io.FileIO(training_dataset_dir, mode='r')
      datas = pd.read_csv(file_stream,iterator=True, chunksize=bs )
      for data in datas:
          data[' pixels'] = data[' pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])
          X, Y = data[' pixels'].tolist(), data['emotion'].values
          X = np.array(X, dtype='float32').reshape(-1,48,48,1)
          X_res = X/255.0
          Y_res = np.zeros((Y.size, 7))
          Y_res[np.arange(Y.size),Y] = 1
          if aug is not None:
              (X_res, Y_res) = next(aug.flow(np.array(X_res),
                  Y_res, batch_size=bs))
          yield  X_res, Y_res

In [0]:
training_dataset_dir = '/content/drive/My Drive/cs230 project/collab/fer2013/train.csv'
dev_dataset_dir = '/content/drive/My Drive/cs230 project/collab/fer2013/dev.csv'

aug = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)



train_generator = get_data(training_dataset_dir,  bs=BS, aug=aug)
dev_generator   = get_data(dev_dataset_dir, bs=BS, aug=None)
    #X_dev_res, Y_dev_res  = get_data(dev_dataset_dir)

# Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches) indefinitely
# rescale:          Rescaling factor (defaults to None). Multiply the data by the value provided (before applying any other transformation)
# rotation_range:   Int. Degree range for random rotations
# shear_range:      Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
# zoom_range:       Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range]
# fill_mode :       Points outside the boundaries of the input are filled according to the given mode: {"constant", "nearest", "reflect" or "wrap"}
# horizontal_flip:  Boolean. Randomly flip inputs horizontally


# Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
#train_generator = train_datagen.flow(X_train_res, Y_train_res,  batch_size  = BS)

In [20]:
history = model.fit_generator(
    generator = train_generator,
    validation_data=dev_generator, 
    steps_per_epoch=28709// BS,
    validation_steps=3509 // BS,
    shuffle=True,
    epochs=EPOCHS,
    callbacks=[rlrop]
) 

Epoch 1/20
224/224 [==============================] - 38s 170ms/step - loss: 1.6165 - acc: 0.3636 - val_loss: 1.5328 - val_acc: 0.4016
Epoch 2/20
224/224 [==============================] - 38s 168ms/step - loss: 1.5542 - acc: 0.3945 - val_loss: 1.5248 - val_acc: 0.4039
Epoch 3/20
224/224 [==============================] - 37s 167ms/step - loss: 1.4998 - acc: 0.4154 - val_loss: 1.4235 - val_acc: 0.4517
Epoch 4/20
224/224 [==============================] - 37s 167ms/step - loss: 1.4536 - acc: 0.4385 - val_loss: 1.3637 - val_acc: 0.4783
Epoch 5/20
224/224 [==============================] - 37s 167ms/step - loss: 1.4095 - acc: 0.4558 - val_loss: 1.3423 - val_acc: 0.4887
Epoch 6/20
224/224 [==============================] - 38s 168ms/step - loss: 1.3780 - acc: 0.4703 - val_loss: 1.2522 - val_acc: 0.5205
Epoch 7/20
224/224 [==============================] - 38s 170ms/step - loss: 1.3371 - acc: 0.4886 - val_loss: 1.2148 - val_acc: 0.5411
Epoch 8/20
224/224 [==============================] - 3

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.103609). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


224/224 [==============================] - 38s 169ms/step - loss: 1.2482 - acc: 0.5225 - val_loss: 1.7463 - val_acc: 0.3388
Epoch 11/20
224/224 [==============================] - 38s 170ms/step - loss: 1.2221 - acc: 0.5353 - val_loss: 1.5509 - val_acc: 0.4152
Epoch 12/20
224/224 [==============================] - 38s 170ms/step - loss: 1.1915 - acc: 0.5452 - val_loss: 1.1071 - val_acc: 0.5825
Epoch 13/20
224/224 [==============================] - 38s 169ms/step - loss: 1.1688 - acc: 0.5567 - val_loss: 1.2422 - val_acc: 0.5295
Epoch 14/20
224/224 [==============================] - 39s 172ms/step - loss: 1.1525 - acc: 0.5636 - val_loss: 1.0336 - val_acc: 0.6105
Epoch 15/20
224/224 [==============================] - 38s 171ms/step - loss: 1.1371 - acc: 0.5690 - val_loss: 0.9631 - val_acc: 0.6429
Epoch 16/20
224/224 [==============================] - 38s 168ms/step - loss: 1.1132 - acc: 0.5789 - val_loss: 1.0011 - val_acc: 0.6291
Epoch 17/20
224/224 [==============================] - 37s 1

In [0]:
print('\n# Evaluate on dev data')
results_dev = model.evaluate(X_dev, Y_dev , batch_size=128)
print('dev loss, dev acc:', results_dev)